# ML-модель для геомаркетинга. В этом ноутбуке показано, как происходили предобработка данных, обучение и тестирование модели.

## Загрузка данных и препроцессинг.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline 
!pip3 install catboost
from catboost import CatBoostRegressor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 10.4 MB/s eta 0:00:00


## Загрузка данных о расположении магазинов на карте (координаты).

In [ ]:
from openpyxl import load_workbook
from pathlib import Path
import itertools as it

id_to_address = [[0, 0]]
ws = load_workbook(Path('market-coordinates.xlsx'))['Sheet1']
address_to_id = {}
for i, row in it.islice(enumerate(ws.rows), 1, 1000):
    id_to_address.append([row[2].value, row[3].value])
    address_to_id[row[1].value] = int(row[0].value)

## Сопоставление адресов и координат.

In [ ]:
ws = load_workbook(Path('filled-table.xlsx'), read_only=True)['Sheet']

market_address_ids = []
market_visitor_frequencies = []

for i, row in it.islice(enumerate(ws.rows), 229):
    address = ' '.join([str(row[i].value) for i in [4, 3, 2]])
    #print(address)
    market_address_ids.append([row[0].value, address_to_id[address]])
    
    visitors = int(row[14].value)
    try:
        days = int(row[15].value)
    except TypeError:
        days = 30
    market_visitor_frequencies.append(visitors / days)

print(market_address_ids)
print(len(market_address_ids))
market_address_ids = np.array(market_address_ids)
market_visitor_frequencies = np.array(market_visitor_frequencies)

[[1, 1], [214, 137], [237, 118], [255, 193], [253, 191], [23, 23], [96, 87], [107, 96], [151, 122], [292, 226], [2, 2], [6, 6], [10, 10], [26, 26], [195, 104], [208, 162], [125, 107], [308, 233], [152, 123], [157, 128], [162, 133], [244, 182], [185, 108], [200, 154], [205, 159], [247, 185], [307, 232], [77, 74], [213, 167], [174, 143], [261, 150], [279, 213], [286, 220], [310, 235], [317, 242], [145, 119], [199, 119], [264, 119], [379, 119], [42, 42], [73, 71], [225, 176], [226, 176], [329, 249], [330, 250], [436, 309], [90, 84], [91, 84], [92, 84], [93, 84], [171, 140], [178, 79], [354, 78], [361, 84], [100, 91], [184, 149], [312, 237], [318, 243], [358, 149], [30, 30], [52, 52], [113, 100], [54, 54], [59, 59], [159, 130], [164, 135], [390, 285], [394, 289], [44, 44], [68, 68], [268, 202], [291, 225], [103, 93], [105, 94], [123, 105], [180, 145], [192, 118], [319, 93], [453, 105], [40, 40], [56, 56], [252, 190], [340, 260], [381, 276], [388, 283], [169, 138], [266, 200], [267, 201], [

## Загрузка основного датасета (информация о площади и посещаемости магазинов).

In [ ]:
main_data = pd.read_csv('./main_data_10.csv')
main_data.head()

,Num,Type,Square,Name,Freqs,Period,Building,FPD
0,1,супермаркет,664,Пятерочка,24 945,28,отдельное здание,"890,8928571"
1,237,магазин,137,Другие,7 417,31,жилой дом,"239,2580645"
2,253,супермаркет,203,Ароматный мир,11 760,31,жилой дом,"379,3548387"
3,23,магазин,216,Другие,6 216,33,жилой дом,"188,3636364"
4,96,магазин,229,Другие,6 002,30,жилой дом,"200,0666667"


### Узнаем размер датасета.

In [ ]:
cntstr = main_data.shape[0]
print(cntstr)

134


### Сопоставление номера магазина в таблице и его уникального ID, к которому привязан адрес.

In [ ]:
id = []
for i in range(cntstr):
  for j in range(229):
    if (main_data['Num'][i] == market_address_ids[j][0]):
      id.append(market_address_ids[j][1])
      break
    j += 1
  i += 1
print(id)
print(len(id))

[1, 118, 191, 23, 87, 122, 2, 6, 10, 26, 107, 233, 128, 133, 182, 108, 159, 185, 232, 167, 220, 235, 242, 119, 42, 176, 249, 250, 309, 84, 84, 84, 140, 79, 91, 237, 243, 30, 100, 54, 59, 44, 68, 202, 94, 145, 105, 56, 260, 283, 138, 200, 201, 216, 229, 109, 111, 126, 257, 258, 270, 168, 318, 117, 155, 304, 117, 37, 69, 188, 5, 12, 73, 268, 206, 147, 275, 115, 91, 221, 222, 224, 152, 152, 161, 89, 269, 61, 58, 67, 101, 114, 151, 114, 27, 43, 203, 120, 251, 297, 3, 98, 94, 137, 219, 36, 183, 256, 277, 192, 291, 38, 196, 234, 204, 34, 252, 265, 266, 148, 85, 232, 112, 280, 281, 284, 290, 298, 300, 230, 85, 306, 315, 150]
134


In [ ]:
main_data.insert(1, 'ID', id)
main_data.head()

,Num,ID,Type,Square,Name,Freqs,Period,Building,FPD
0,1,1,супермаркет,664,Пятерочка,24 945,28,отдельное здание,"890,8928571"
1,237,118,магазин,137,Другие,7 417,31,жилой дом,"239,2580645"
2,253,191,супермаркет,203,Ароматный мир,11 760,31,жилой дом,"379,3548387"
3,23,23,магазин,216,Другие,6 216,33,жилой дом,"188,3636364"
4,96,87,магазин,229,Другие,6 002,30,жилой дом,"200,0666667"


### Считываем информацию о времени в пути от каждого из магазинов до каждого из районов.

In [ ]:
dists = pd.read_csv('./fixed_dist.csv')
dists.head()

,District,Population,Store1,Store2,Store3,Store4,Store5,Store6,Store7,Store8,...,Store309,Store310,Store311,Store312,Store313,Store314,Store315,Store316,Store317,Store318
0,1,646,"28,05127469","40,54699233","39,86133175","29,96589644","29,08400242","35,20199631","32,16069402","30,76793155",...,"37,15582635","12,52727614","31,92653471","8,757369081","20,04190608","3,463823592","15,86386368","31,81204067","22,17310407","22,17310407"
1,2,1562,"24,53101023","36,51983572","38,76202941","25,93873983","26,49269262","33,65528252","28,13353742","26,74077495",...,"36,05652401","18,29523877","37,17121656","10,29062559","23,30607539","8,138245136","17,32035857","37,58000329","23,72590979","23,72590979"
2,3,333,"20,18574414","23,90631289","34,28788331","14,26139268","19,63058031","23,32137173","17,35291896","15,98574574",...,"32,56289287","44,25954175","62,11198903","32,39767337","45,3715918","36,53470906","39,33686203","62,90653109","45,30680316","45,30680316"
3,4,1984,"11,75641437","19,64331217","26,31888014","8,41871025","11,22391948","17,20739571","11,30508437","9,912321898",...,"24,1335631","36,3457885","53,68265926","28,81623836","36,94226203","28,82847446","35,75542701","54,47720132","36,87747339","36,87747339"
4,5,2007,"13,89782815","14,75511342","23,47769306","3,444759625","13,34266433","12,50473867","6,557232515","5,169112684",...,"26,27497689","38,48720229","55,82407304","33,5081258","39,08367582","31,46759986","40,44731446","56,61861511","39,01888717","39,01888717"


### Создадим вспомогательные функции для расчета по формуле Хаффа и обработки ошибок в данных

In [ ]:
def to_flt(s):
  if type(s) != type('help'):
    return s
  s = s.replace(' ', '')
  s = s.replace('\xa0', '')
  s = s.split(',')
  s = '.'.join(s)
  return float(s)

def prob_counter(alp, lamb, i, j): # i - номер района, j - номер магазина (порядковый, не ID !!!)
  sum = 0
  for v in range(cntstr):
    distutil = to_flt((dists[ ('Store' + str(main_data['ID'][v]) )][i - 1]))
    sum += to_flt(main_data['Square'][v])**alp / (distutil if distutil < 25 else distutil/5)**lamb
  distutil = to_flt((dists[ ('Store' + str(main_data['ID'][j - 1]) )][i - 1]))
  return (dists['Population'][i - 1]) * to_flt(main_data['Square'][j - 1])**alp / (distutil if distutil < 25 else distutil/5)**lamb / sum


### Добавляем в основные данные столбец с предполагаемой посещаемостью (по Хаффу).

In [ ]:
main_data.insert(7, 'Huff_predict', list(i for i in range(cntstr)))
main_data.head()

,Num,ID,Type,Square,Name,Freqs,Period,Huff_predict,Building,FPD
0,1,1,супермаркет,664,Пятерочка,24 945,28,0,отдельное здание,"890,8928571"
1,237,118,магазин,137,Другие,7 417,31,1,жилой дом,"239,2580645"
2,253,191,супермаркет,203,Ароматный мир,11 760,31,2,жилой дом,"379,3548387"
3,23,23,магазин,216,Другие,6 216,33,3,жилой дом,"188,3636364"
4,96,87,магазин,229,Другие,6 002,30,4,жилой дом,"200,0666667"


### Создаем функцию для вычисления посещаемости по формуле Хаффа.

In [ ]:
def huff_predict(i, lamb, alp):
  sum = 0
  for j in range(112):
    sum += prob_counter(alp, lamb, j + 1, i + 1)
  return sum

### Разделяем столбцы.

In [ ]:
cat_cols  = ['Type', 'Name', 'Building']
num_cols = ['Square', 'Huff_predict']
target_col = 'FPD'

### Заполняем столбец в основном датасете подсчитанными по формуле Хаффа значениями предполагаемой посещаемости.

In [ ]:
lamb = 0.8
alp = 1
for i in range(cntstr):
  main_data['Huff_predict'][i] = huff_predict(i, lamb, alp)
  main_data['Square'][i] = to_flt(main_data['Square'][i]) 
main_data.head()

<ipython-input-13-5f0c9b7eac86>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_data['Huff_predict'][i] = huff_predict(i, lamb, alp)
<ipython-input-13-5f0c9b7eac86>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_data['Square'][i] = to_flt(main_data['Square'][i])


,Num,ID,Type,Square,Name,Freqs,Period,Huff_predict,Building,FPD
0,1,1,супермаркет,664.0,Пятерочка,24 945,28,3507.421614,отдельное здание,"890,8928571"
1,237,118,магазин,137.0,Другие,7 417,31,702.405165,жилой дом,"239,2580645"
2,253,191,супермаркет,203.0,Ароматный мир,11 760,31,1043.342291,жилой дом,"379,3548387"
3,23,23,магазин,216.0,Другие,6 216,33,1070.613756,жилой дом,"188,3636364"
4,96,87,магазин,229.0,Другие,6 002,30,1438.618229,жилой дом,"200,0666667"


### Удаляем лишние столбцы

In [ ]:
X = main_data.drop(target_col, axis = 1)
X = X.drop('Num', axis = 1)
X = X.drop('ID', axis = 1)
#X = X.drop('Square', axis = 1)
X = X.drop('Freqs', axis = 1)
X = X.drop('Period', axis = 1)
#X = X.drop('Building', axis = 1)
#X = X.drop('Type', axis = 1)
X = X.astype({'Square': np.float})
X.head()

<ipython-input-81-290cb514d224>:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = X.astype({'Square': np.float})


,Type,Square,Name,Huff_predict,Building
0,супермаркет,664.0,Пятерочка,3507.421614,отдельное здание
1,магазин,137.0,Другие,702.405165,жилой дом
2,супермаркет,203.0,Ароматный мир,1043.342291,жилой дом
3,магазин,216.0,Другие,1070.613756,жилой дом
4,магазин,229.0,Другие,1438.618229,жилой дом


In [ ]:
Y = main_data[target_col]
for i in range(len(Y)):
  Y[i] = to_flt(Y[i])
Y = Y.astype({'FPD' : np.float})
Y.head

<ipython-input-54-1a38e3094e10>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y[i] = to_flt(Y[i])
<ipython-input-54-1a38e3094e10>:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y = Y.astype({'FPD' : np.float})


<bound method NDFrame.head of 0      890.892857
1      239.258064
2      379.354839
3      188.363636
4      200.066667
          ...    
129    473.548387
130    159.241379
131    215.156250
132    223.909091
133    293.033333
Name: FPD, Length: 134, dtype: float64>

### Создаем, обучаем и тестируем модель.

In [ ]:
X = pd.get_dummies(X, columns=cat_cols)


In [ ]:
import torch
!pip install torchmetrics
from torch import nn
from torch.nn import MSELoss, L1Loss
from torchmetrics import MeanAbsolutePercentageError


torch.manual_seed(17)


class NeuralNetwork(torch.nn.Module):

    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(len(X.columns), 64),
            nn.LeakyReLU(0.1),
            nn.Linear(64, 32),
            nn.LeakyReLU(0.1),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        y_pred = self.layers(x)
        return y_pred

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
X.dtypes

Square                       float64
Huff_predict                 float64
Type_дискаунтер                uint8
Type_киоск                     uint8
Type_лавка                     uint8
Type_магазин                   uint8
Type_минимаркет                uint8
Type_супермаркет               uint8
Name_Азбука Вкуса              uint8
Name_Ароматный мир             uint8
Name_ВкусВилл                  uint8
Name_Дикси                     uint8
Name_Другие                    uint8
Name_Красное & Белое           uint8
Name_Лента                     uint8
Name_Магнит                    uint8
Name_Перекресток               uint8
Name_Пятерочка                 uint8
Name_Семишагофф                uint8
Name_Фасоль                    uint8
Building_ТЦ                    uint8
Building_жилой дм              uint8
Building_жилой дом             uint8
Building_жилой дом             uint8
Building_отдельное здание      uint8
Building_павильон              uint8
dtype: object

In [ ]:
m_model = NeuralNetwork().to(device)
optimizer_ln = torch.optim.Adam(m_model.parameters(), lr=0.008)


# choose loss
loss_l1 = L1Loss()
loss_mse = MSELoss()
loss_mape = MeanAbsolutePercentageError()
loss = loss_l1
from torch.autograd import Variable
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15)

x_data_train = Variable(torch.Tensor(X_train.values)).to(device)
y_data_train = Variable(torch.Tensor([[target] for target in Y_train.values])).to(device)

for epoch in range(20000):
    pred_y = m_model(x_data_train)
    loss_ln = loss(pred_y, y_data_train)

    optimizer_ln.zero_grad()
    loss_ln.backward()
    optimizer_ln.step()

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import StratifiedKFold

In [ ]:
x_data_test = Variable(torch.Tensor(X_test.values)).to(device)
y_data_test = Variable(torch.Tensor([[target] for target in Y_test.values])).to(device)
x_data_all = Variable(torch.Tensor(X.values)).to(device)
Y_pred = m_model(x_data_test)
Y_predict = m_model(x_data_all)

aim = mean_absolute_percentage_error(Y, Y_predict.cpu().detach().numpy())
print('\nMAPE на всем датасете:', aim)
print('\nMAPE на закрытой тестовой выборке:', mean_absolute_percentage_error(Y_test, Y_pred.cpu().detach().numpy()))


MAPE на всем датасете: 0.20548084305889325

MAPE на закрытой тестовой выборке: 0.547046375150979


##Сохранение модели

In [ ]:
torch.save(m_model.state_dict(), 'model_pars')

##Скачивание модели
Тут надо снова создать класс (можно назвать по-другому, не важно). Главное - полное совпадение слоев.

Нетрудно видеть, что после загрузки результаты предсказаний такие же, что и у оригинальной модели. Значит, все работает.

In [ ]:
class NeuralNetwork_copy(torch.nn.Module):
    def __init__(self):
        super(NeuralNetwork_copy, self).__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(len(X.columns), 64),
            nn.LeakyReLU(0.1),
            nn.Linear(64, 32),
            nn.LeakyReLU(0.1),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        y_pred = self.layers(x)
        return y_pred


loaded_model = NeuralNetwork_copy()
loaded_model.load_state_dict(torch.load('model_pars'))
loaded_model.eval()

Y_pred_1 = m_model(x_data_test)
Y_predict_1 = m_model(x_data_all)

aim = mean_absolute_percentage_error(Y, Y_predict_1.cpu().detach().numpy())
print('\nMAPE на всем датасете:', aim)
print('\nMAPE на закрытой тестовой выборке:', mean_absolute_percentage_error(Y_test, Y_pred_1.cpu().detach().numpy()))


MAPE на всем датасете: 0.20548084305889325

MAPE на закрытой тестовой выборке: 0.547046375150979


### Как мы видим, на всём датасете MAPE получилась порядка 20.55%, что немного лучше, чем аналогичный показатель у модели на catboost.